In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from skimage.morphology import disk
from skimage.filters import rank
from skimage.util.dtype import dtype_range
from skimage import exposure
import dask_image.imread
from skimage.filters import threshold_multiotsu
from cellpose import models
import napari
import tifffile
import dask.array as da

## Plotting Function

In [2]:
def plot_image(img):
    plt.figure(figsize=(20, 20)) 
    plt.imshow(img)
    plt.axis('off')
    plt.show()

## Image processing for full segmentation

In [ ]:
def normalize_chunk(chunk):
    kernel = np.ones((5, 5), np.uint8) 
    chunk_erode = cv2.erode(chunk, kernel, iterations=3)       
    footprint = disk(100)
    polyt_eq = rank.equalize(chunk_erode, footprint=footprint)
    return polyt_eq

# Load the image as a Dask array
polyt_image = dask_image.imread.imread('raw/mosaic_PolyT_z0.tif')[0]

chunks = (7555, 8881)
polyt_image = polyt_image.rechunk(chunks)

# Apply the normalization function to each chunk
polyt = polyt_image.map_overlap(normalize_chunk, depth=200, trim=True, chunks=chunks)

In [ ]:
#tifffile.imwrite('polyt_eq.tif', polyt.compute())

In [ ]:
def erode_chunk(chunk):
    kernel = np.ones((5, 5), np.uint8) 
    chunk_erode = cv2.erode(chunk, kernel, iterations=5)       
    return chunk_erode

# Load the image as a Dask array
dapi_image = dask_image.imread.imread('raw/mosaic_DAPI_z0.tif')[0]

chunks = (7555, 8881)
dapi_image = dapi_image.rechunk(chunks)

# Apply the normalization function to each chunk
dapi = dapi_image.map_overlap(erode_chunk, depth=200, trim=True, chunks=chunks)

In [ ]:
#tifffile.imwrite('dapi_erode.tif', dapi.compute())

## Cellpose on crop

In [ ]:
filename_pattern = 'raw/*.tif'
images = dask_image.imread.imread(filename_pattern)

In [ ]:
polyt = images[1, 15000:20000,15000:20000].compute()
nuclei = imagesnuclei = images[0, 15000:20000,15000:20000].compute()

In [ ]:
kernel = np.ones((5, 5), np.uint8) 
polyt_erode = cv2.erode(polyt, kernel, iterations=3)
nuclei_erode = cv2.erode(nuclei, kernel, iterations=5)
blank = np.zeros(nuclei.shape)

In [ ]:
# Equalization
footprint = disk(100)
polyt_eq = rank.equalize(polyt_erode, footprint=footprint)

In [ ]:
stacked_image = np.stack((blank, polyt_eq, nuclei_erode), axis=0)

In [ ]:
model = models.Cellpose(gpu=True, model_type='cyto3')
masks, flows, styles, diams = model.eval([stacked_image], diameter=130, channels=[2,3], flow_threshold=0.9, do_3D=False)